In [1]:
from copy import deepcopy
from time import time

import torch
import torch.nn.functional as F

from torch import nn
from torch.utils.data import DataLoader, TensorDataset

import numpy as np
import pandas as pd

from torchvision.datasets import FashionMNIST
from torchvision import transforms

from ray import tune

/home/patryk/.local/share/virtualenvs/lunar-landing-q6dpKwgz/lib/python3.7/site-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  "update your install command.", FutureWarning)


In [2]:
def acc(net_output, labels):
    predicted = net_output.argmax(dim=1)
    correct = (predicted == labels).sum()
    examples = len(labels)
    return (correct / examples).item()


class FMnistNet(nn.Module):
    def __init__(self, l1_size, l2_size):
        super().__init__()
        self.fc1 = nn.Linear(784, l1_size)
        self.fc2 = nn.Linear(l1_size, l2_size)
        self.fc3 = nn.Linear(l2_size, 10)
        
    def forward(self, x):
        x = x.reshape(-1, 28*28)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.softmax(self.fc3(x), dim=1)
        return x
    
    
def training_function(config):
    lr, batch_size, l1_size, l2_size = config['lr'], config['batch_size'], config['l1_size'], config['l2_size']
    
    fmnist_transforms = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(0, 1)
        ])
    
    ds_train = FashionMNIST(
        root=FMNIST_DATA_DIR, 
        transform=fmnist_transforms,
        download=True, 
        train=True)

    ds_test = FashionMNIST(
        root=FMNIST_DATA_DIR, 
        transform=fmnist_transforms, 
        download=True,
        train=False)

    net = FMnistNet(l1_size, l2_size)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(net.parameters(), lr=lr)
    
    train_set = DataLoader(ds_train, batch_size=batch_size, shuffle=True, num_workers=4)
    test_set = DataLoader(ds_test, batch_size=len(ds_train))

    
    for epoch in range(EPOCHS_PER_RUN):
        for data, labels in train_set:
            optimizer.zero_grad()
            net_output = net(data)
            loss = loss_fn(net_output, labels)
            loss.backward()
            optimizer.step()

    test_data, test_labels = next(iter(test_set))
    with torch.no_grad():
        test_output = net(test_data)
        test_acc = acc(test_output, test_labels)     
        tune.report(test_acc=test_acc)

In [3]:
EPOCHS_PER_RUN=5
FMNIST_DATA_DIR='~/Documents/lunar-landing/src/fmnist/data'

In [4]:
analysis = tune.run(
    training_function,
    config={
        "lr": tune.grid_search([0.001, 0.01, 0.1]),
        "batch_size": tune.grid_search([32, 64]),
        "l1_size": tune.grid_search([64, 128, 256]),
        "l2_size": tune.grid_search([64, 128, 256]),
    },
    local_dir='./fmnist_logs',
    verbose=False
)

2021-07-30 11:47:53,483	INFO services.py:1247 -- View the Ray dashboard at http://127.0.0.1:8265
2021-07-30 11:47:55,171	WARNING function_runner.py:545 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.
2021-07-30 11:47:55,253	WARNING tune.py:495 -- Tune detects GPUs, but no trials are using GPUs. To enable trials to use GPUs, set tune.run(resources_per_trial={'gpu': 1}...) which allows Tune to expose 1 GPU to each trial. You can also override `Trainable.default_resource_request` if using the Trainable API.
(pid=189973) /home/patryk/.local/share/virtualenvs/lunar-landing-q6dpKwgz/lib/python3.7/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumP

(pid=190752) /home/patryk/.local/share/virtualenvs/lunar-landing-q6dpKwgz/lib/python3.7/site-packages/torch/autograd/__init__.py:149: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:115.)
(pid=190752)   allow_unreachable=True, accumulate_grad=True)  # allow_unreachable flag
(pid=190750) /home/patryk/.local/share/virtualenvs/lunar-landing-q6dpKwgz/lib/python3.7/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for

(raylet) /home/patryk/.local/share/virtualenvs/lunar-landing-q6dpKwgz/lib/python3.7/site-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
(raylet)   "update your install command.", FutureWarning)
(raylet) /home/patryk/.local/share/virtualenvs/lunar-landing-q6dpKwgz/lib/python3.7/site-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
(raylet)   "update your install command.", FutureWarning)
(raylet) /home/patryk/.local/share/virtualenvs/lunar-landing-q6dpKwgz/lib/python3.7/site-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+,

(pid=192573) /home/patryk/.local/share/virtualenvs/lunar-landing-q6dpKwgz/lib/python3.7/site-packages/torch/autograd/__init__.py:149: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:115.)
(pid=192573)   allow_unreachable=True, accumulate_grad=True)  # allow_unreachable flag
(raylet) /home/patryk/.local/share/virtualenvs/lunar-landing-q6dpKwgz/lib/python3.7/site-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
(raylet)   "update your install command.", FutureWarning)
(raylet) /home/patryk/.local/share/virtualenvs/lunar-landing-q6dpKwgz/lib/python3.7/site-packag

(pid=193463) /home/patryk/.local/share/virtualenvs/lunar-landing-q6dpKwgz/lib/python3.7/site-packages/torch/autograd/__init__.py:149: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:115.)
(pid=193463)   allow_unreachable=True, accumulate_grad=True)  # allow_unreachable flag
(raylet) /home/patryk/.local/share/virtualenvs/lunar-landing-q6dpKwgz/lib/python3.7/site-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
(raylet)   "update your install command.", FutureWarning)
(raylet) /home/patryk/.local/share/virtualenvs/lunar-landing-q6dpKwgz/lib/python3.7/site-packag

(pid=193969) Traceback (most recent call last):
(pid=193969)   File "/usr/lib/python3.7/multiprocessing/resource_sharer.py", line 142, in _serve
(pid=193969)     with self._listener.accept() as conn:
(pid=193969)   File "/usr/lib/python3.7/multiprocessing/connection.py", line 455, in accept
(pid=193969)     deliver_challenge(c, self._authkey)
(pid=193969)   File "/usr/lib/python3.7/multiprocessing/connection.py", line 730, in deliver_challenge
(pid=193969)     response = connection.recv_bytes(256)        # reject large message
(pid=193969)   File "/usr/lib/python3.7/multiprocessing/connection.py", line 216, in recv_bytes
(pid=193969)     buf = self._recv_bytes(maxlength)
(pid=193969)   File "/usr/lib/python3.7/multiprocessing/connection.py", line 407, in _recv_bytes
(pid=193969)     buf = self._recv(4)
(pid=193969)   File "/usr/lib/python3.7/multiprocessing/connection.py", line 379, in _recv
(pid=193969)     chunk = read(handle, remaining)
(pid=193969) ConnectionResetError: [Errno 104]

In [5]:
print("Best config: ", analysis.get_best_config(
    metric="test_acc", mode="max")
     )

# Get a dataframe for analyzing trial results.
df = analysis.results_df

Best config:  {'lr': 0.1, 'batch_size': 32, 'l1_size': 128, 'l2_size': 64}


2021-07-30 09:09:37,838	WARNING worker.py:1189 -- The autoscaler failed with the following error:
Traceback (most recent call last):
  File "/home/patryk/.local/share/virtualenvs/lunar-landing-q6dpKwgz/lib/python3.7/site-packages/ray/autoscaler/_private/monitor.py", line 317, in run
    self._run()
  File "/home/patryk/.local/share/virtualenvs/lunar-landing-q6dpKwgz/lib/python3.7/site-packages/ray/autoscaler/_private/monitor.py", line 207, in _run
    self.update_load_metrics()
  File "/home/patryk/.local/share/virtualenvs/lunar-landing-q6dpKwgz/lib/python3.7/site-packages/ray/autoscaler/_private/monitor.py", line 170, in update_load_metrics
    request, timeout=4)
  File "/home/patryk/.local/share/virtualenvs/lunar-landing-q6dpKwgz/lib/python3.7/site-packages/grpc/_channel.py", line 946, in __call__
    return _end_unary_response_blocking(state, call, False, None)
  File "/home/patryk/.local/share/virtualenvs/lunar-landing-q6dpKwgz/lib/python3.7/site-packages/grpc/_channel.py", line 8

In [6]:
tune.run?